<a href="https://colab.research.google.com/github/ahmetemincopur/ImageProcessing_UniversityFinalProject/blob/main/UniversityFinalProject_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/My Drive/Colab Notebooks/4.2.Dönem-Bitirme Projesi
%ls

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/4.2.Dönem-Bitirme Projesi
angles2.csv  binary-model1.h5                      modeller/
angles3.csv  BitirmeProje.ipynb                    test2.csv
angles.csv   BitirmeProjesiModellerleTahmin.ipynb  test.csv


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
import numpy as np
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

#numpy.random.seed(7)
#dataset=numpy.loadtxt('angles2.csv' , delimiter='|')
import pandas as pd

data = pd.read_csv('angles.csv', delimiter='|')
dataset = data.iloc[:, 0:12].values

datalar=dataset[:,0:11]
etiketler=dataset[:,11]

In [ ]:
from sklearn.model_selection import train_test_split

# Veri setini eğitim ve test setlerine ayır
train_datalar, test_data, train_etiketler, test_etiketler = train_test_split(datalar, etiketler, test_size=0.1, random_state=42)

# Eğitim setini eğitim ve doğrulama setlerine ayır
train_data, validation_data, train_etiketler, validation_etiketler = train_test_split(train_datalar, train_etiketler, test_size=0.111, random_state=42)

train_etiketler = train_etiketler.astype('int32')
validation_etiketler = validation_etiketler.astype('int32')
test_etiketler = test_etiketler.astype('int32')

print(train_data[0])

train_datalar=train_data[:,1:11]
test_datalar=test_data[:,1:11]
validation_datalar=validation_data[:,1:11]

print(train_datalar[0])

train_datalar = tf.convert_to_tensor(train_datalar, dtype=tf.float32)
test_datalar = tf.convert_to_tensor(test_datalar, dtype=tf.float32)
validation_datalar = tf.convert_to_tensor(validation_datalar, dtype=tf.float32)

In [ ]:
model=Sequential()
model.add(Dense(256, activation='relu', input_dim=10))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

basarim=model.evaluate(train_datalar, train_etiketler)
print("\n %s : %.2f%%" % (model.metrics_names[1], basarim[1]*100))
print("------------------------------------------------------")
basarim=model.evaluate(train_datalar, train_etiketler)
print("\n %s : %.2f%%" % (model.metrics_names[0], basarim[0]*100))

237/237 [==============================] - 1s 3ms/step - loss: 16.1792 - accuracy: 0.3030

 accuracy : 30.30%
------------------------------------------------------
237/237 [==============================] - 1s 3ms/step - loss: 16.1792 - accuracy: 0.3030

 loss : 1617.92%


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

best_models_info = []

# Gizli katmanların farklı konfigürasyonlarını tanımla
hidden_layer_configs = [
    [128, layer3_units] for layer3_units in range(1, 64)
]

best_val_accuracy = 0
best_val_loss = float('inf')
best_train_accuracy = 0
best_train_loss = float('inf')
best_model = None
best_hidden_layers = []

for hidden_layers in hidden_layer_configs:
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=10))
    for num_units in hidden_layers:
        model.add(keras.layers.Dense(num_units, activation='relu'))
    model.add(keras.layers.Dense(3, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(train_datalar, train_etiketler, epochs=10, batch_size=32, validation_data=(validation_datalar, validation_etiketler))
    val_accuracy = history.history['val_accuracy'][-1]
    val_loss = history.history['val_loss'][-1]

    train_accuracy = history.history['accuracy'][-1]
    train_loss = history.history['loss'][-1]

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_val_loss = val_loss
        best_train_accuracy = train_accuracy
        best_train_loss = train_loss
        best_model = model
        best_hidden_layers = hidden_layers

    # Her modelin ayrıntılarını sakla
    model_info = {
        'hidden_layers': hidden_layers,
        'val_accuracy': val_accuracy,
        'val_loss': val_loss,
        'train_accuracy': train_accuracy,
        'train_loss': train_loss
    }
    best_models_info.append(model_info)

# En iyi modelin ayrıntılarını yazdır
print("Best Model Architecture:")
print(best_model.summary())
print("Best Model Hyperparameters:")
print("Hidden Layers:", best_hidden_layers)
print("Validation Accuracy:", best_val_accuracy)
print("Validation Loss:", best_val_loss)
print("Train Accuracy:", best_train_accuracy)
print("Train Loss:", best_train_loss)

# Bütün modellerin ayrıntılarını yazdır
print("\nDetails of All Models:")
for model_info in best_models_info:
    print("\nHidden Layers:", model_info['hidden_layers'])
    print("Validation Accuracy:", model_info['val_accuracy'])
    print("Validation Loss:", model_info['val_loss'])
    print("Train Accuracy:", model_info['train_accuracy'])
    print("Train Loss:", model_info['train_loss'])
    print("--------------------------------------------------")


In [ ]:
print(best_model)

In [ ]:
import matplotlib.pyplot as plt

# Modelin eğitim ve doğrulama kayıplarını ve doğruluk değerlerini elde et
history = model.fit(train_datalar, train_etiketler, epochs=100, batch_size=32, validation_data=(validation_datalar, validation_etiketler))

# Eğitim ve doğrulama kaybı değerlerini al
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Eğitim ve doğrulama doğruluk değerlerini al
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Eğitim ve doğrulama kayıp değerlerinin grafiğini çiz
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_loss) + 1), train_loss, label='Eğitim Kaybı')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Doğrulama Kaybı')
plt.title('Eğitim ve Doğrulama Kaybı')
plt.xlabel('Epok')
plt.ylabel('Kayıp')
plt.legend()

# Eğitim ve doğrulama doğruluk değerlerinin grafiğini çiz
plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_acc) + 1), train_acc, label='Eğitim Doğruluğu')
plt.plot(range(1, len(val_acc) + 1), val_acc, label='Doğrulama Doğruluğu')
plt.title('Eğitim ve Doğrulama Doğruluğu')
plt.xlabel('Epok')
plt.ylabel('Doğruluk')
plt.legend()

# Grafikleri göster
plt.tight_layout()
plt.show()


In [ ]:
basarim=model.evaluate(train_datalar, train_etiketler)
print("\n %s : %.2f%%" % (model.metrics_names[1], basarim[1]*100))
print("------------------------------------------------------")
basarim=model.evaluate(train_datalar, train_etiketler)
print("\n %s : %.2f%%" % (model.metrics_names[0], basarim[0]*100))

print("##########################################################")
basarim=model.evaluate(validation_datalar, validation_etiketler)
print("\n %s : %.2f%%" % (model.metrics_names[1], basarim[1]*100))
print("------------------------------------------------------")
basarim=model.evaluate(validation_datalar, validation_etiketler)
print("\n %s : %.2f%%" % (model.metrics_names[0], basarim[0]*100))

In [ ]:
model.summary()

In [ ]:
# Tahminleri yap
y_pred = model.predict(test_datalar)

# Tahminleri sınıflara dönüştür
y_pred_classes = np.argmax(y_pred, axis=1)

# Gerçek etiketleri sınıflara dönüştür
test_etiketler_classes = np.array(test_etiketler, dtype=int)

# Tahminleri ve gerçek etiketleri karşılaştır ve doğruluğu hesapla
dogruluk = np.mean(y_pred_classes == test_etiketler_classes)

for i in range(len(test_etiketler)):
    resim_adı = test_data[i, 0]  # Burada test_data'yı kullanarak resim adını alıyoruz
    tahmin = y_pred_classes[i]
    print("Resim Adı: %s, Tahmin: %d" % (resim_adı, tahmin))

# Sonucu yazdır
print("Test Doğruluğu: %.2f%%" % (dogruluk * 100))

In [ ]:
from sklearn.metrics import confusion_matrix

# Confusion matrix'i hesapla
cm = confusion_matrix(test_etiketler, y_pred_classes)

# Doğru ve yanlış tahmin sayılarını elde et
true_positive = cm[1, 1]  # Doğru pozitif
true_negative = cm[0, 0]  # Doğru negatif
false_positive = cm[0, 1]  # Yanlış pozitif
false_negative = cm[1, 0]  # Yanlış negatif

print("Doğru Pozitif: ", true_positive)
print("Doğru Negatif: ", true_negative)
print("Yanlış Pozitif: ", false_positive)
print("Yanlış Negatif: ", false_negative)


In [ ]:
model.save("en-uygun-model2.h5")